# Data Analytics Capstone Project
### Beth Harvey
### October 22, 2023

# Introduction

As climate change, habitat loss, and disease continue to threaten wildlife throughout the world, detailed research is becoming more and more important. In 2023 alone, the U.S. Fish and Wildlife service took 21 species off of the Endangered Species Act list because they are now considered extinct [1]. Ten of those 21 species are birds. More research and resources are needed to develop effective conservation efforts. 

Some key components of bird conservation research is being able to accurately classify individual specimens, as well as being able to identify potential connections between different species. One of the most valuable resources for an effort like this is an extensive collection of detailed measurements for a wide variety of species.

## Project Goals

The end goal of this project is to build a model that can predict what taxonomic order a bird falls into based on several physical measurements. This could help researchers more accurately classify individuals when two or more species are possible. The model will be trained and tested using the AVONET dataset [3], a collective effort by researchers and volunteers to provide measurements of over 90,000 specimens of over 11,000 bird species. 

## Data

The data used for this project is from a large, collective effort known as the AVONET dataset. It is a collection of physical measurements, habitat information, and geographic information for over 90,000 specimens representing over 11,000 species of birds [3]. The majority of individuals measured were museum specimens, though many live birds were included as well. The raw data can be downloaded as Microsoft Excel files from https://figshare.com/s/b990722d72a26b5bfead [2]. The physical measurements are described in the Metadata sheet of Supplementary dataset 1 from the link above, and they are also shown in the figure below [3].

![Illustration of bird showing measurements used](measurements-fig.jpg)

The measurements are:

   1. Beak length from tip to base of skull
   2. Beak length from front edge of nostril to tip
   3. Beak width from front edges of nostrils
   4. Beak depth from front edges of nostrils
   5. Lower leg (tarsus) length
   6. Wing length from bend to end of longest feather 
   7. Wing length from bend to end of outer secondary (shorter) feather
   8. Length from tip of outer secondary (shorter) feather to tip of longest feather
   9. Tail length [3]


The remaining measurements are the mass of the bird and the "hand-wing index," which is calculated using measurements 6 and 8. Information about the bird's habitat, migration pattern(s), eating habits, conservation status, and geographic location are also included [2]. The physical measurements and age, sex, and country are all from the AVONET Raw Data sheet of Supplemental dataset 1, and the geographic and behavior information come from the AVONET3 BirdTree sheet of the same file [2]. The categorical information from the BirdTree sheet was matched to the appropriate raw data record to create the additional features. This sheet was chosen over the AVONET1 BirdLife and AVONET2 eBird sheets because it had a match for 90,303 of the 90,372 raw records, which was the highest of the three. The physical measurements shown in the figure above and the mass and hand-wing index are the key features used for this model.

In [1]:
# Import modules
import pandas as pd
from sklearn.impute import KNNImputer
import category_encoders as ce

In [2]:
# Import data
avonet_raw = pd.read_csv('avonet_raw.csv')
birdtree = pd.read_csv('avonet3_birdtree.csv')

# Data Inspection and Cleaning

### Merging Datasets

In [3]:
# Get desired additional features from BirdTree file
birdtree_features = birdtree[['Species3', 'Family3', 'Order3', 'Habitat', 'Habitat.Density', 'Migration', 
                              'Trophic.Level', 'Trophic.Niche','Primary.Lifestyle', 'Centroid.Latitude', 
                             'Centroid.Longitude']]

# Rename column to match avonet_raw column for merging
birdtree_features = birdtree_features.rename(columns = {'Species3': 'Species3_BirdTree', 'Family3': 'Family',
                                                       'Order3': 'Order'})

In [4]:
# Merge dataframes on the Avibase ID
avonet = pd.merge(avonet_raw, birdtree_features, how = 'inner', on = 'Species3_BirdTree')
avonet.head()

,Avibase.ID,Species1_BirdLife,Species2_eBird,eBird.species.group,Species3_BirdTree,Data.type,Source,Specimen.number,Sex,Age,...,Family,Order,Habitat,Habitat.Density,Migration,Trophic.Level,Trophic.Niche,Primary.Lifestyle,Centroid.Latitude,Centroid.Longitude
0,AVIBASE-B3F5E5E2,Abeillia abeillei,Abeillia abeillei,Abeillia abeillei,Abeillia abeillei,1.0,NHMUK,11.4.1890,F,0.0,...,Trochilidae,Apodiformes,Forest,2.0,1.0,Herbivore,Nectarivore,Aerial,15.29,-89.79
1,AVIBASE-B3F5E5E2,Abeillia abeillei,Abeillia abeillei,Abeillia abeillei,Abeillia abeillei,1.0,NHMUK,12.23.1890,F,0.0,...,Trochilidae,Apodiformes,Forest,2.0,1.0,Herbivore,Nectarivore,Aerial,15.29,-89.79
2,AVIBASE-B3F5E5E2,Abeillia abeillei,Abeillia abeillei,Abeillia abeillei,Abeillia abeillei,1.0,NHMUK,1913.3.20.972,M,0.0,...,Trochilidae,Apodiformes,Forest,2.0,1.0,Herbivore,Nectarivore,Aerial,15.29,-89.79
3,AVIBASE-B3F5E5E2,Abeillia abeillei,Abeillia abeillei,Abeillia abeillei,Abeillia abeillei,1.0,NHMUK,87.3.22.1574,M,0.0,...,Trochilidae,Apodiformes,Forest,2.0,1.0,Herbivore,Nectarivore,Aerial,15.29,-89.79
4,AVIBASE-B3F5E5E2,Abeillia abeillei,Abeillia abeillei,Abeillia abeillei,Abeillia abeillei,1.0,NHMUK,NaN,M,0.0,...,Trochilidae,Apodiformes,Forest,2.0,1.0,Herbivore,Nectarivore,Aerial,15.29,-89.79


### Inspect Dataset

In [5]:
# View shape and size of dataset
avonet.shape

(90303, 36)

In [6]:
avonet.describe(include='all')

,Avibase.ID,Species1_BirdLife,Species2_eBird,eBird.species.group,Species3_BirdTree,Data.type,Source,Specimen.number,Sex,Age,...,Family,Order,Habitat,Habitat.Density,Migration,Trophic.Level,Trophic.Niche,Primary.Lifestyle,Centroid.Latitude,Centroid.Longitude
count,90303,89966,89939,90122,90303,89963.000000,89956,73736,89963,89963.000000,...,90303,90303,90140,90266.000000,90260.000000,90303,90298,90303,89282.000000,89282.000000
unique,11207,10990,10495,11198,9988,NaN,80,71809,3,NaN,...,194,40,12,NaN,NaN,4,10,5,NaN,NaN
top,AVIBASE-F9D288FC,Zosterops chloris,Zosterops flavissimus,Zosterops flavissimus,Zosterops chloris,NaN,NHMUK,NOLABEL,M,NaN,...,Tyrannidae,Passeriformes,Forest,NaN,NaN,Carnivore,Invertivore,Insessorial,NaN,NaN
freq,565,784,565,565,784,NaN,46050,102,45495,NaN,...,4760,63634,54298,NaN,NaN,49180,46260,57801,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,1.157587,NaN,NaN,NaN,0.014106,...,NaN,NaN,NaN,1.690637,1.290106,NaN,NaN,NaN,3.031696,8.243624
std,NaN,NaN,NaN,NaN,NaN,0.364355,NaN,NaN,NaN,0.117928,...,NaN,NaN,NaN,0.750236,0.632359,NaN,NaN,NaN,22.077011,85.614841
min,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,0.000000,...,NaN,NaN,NaN,1.000000,1.000000,NaN,NaN,NaN,-71.040000,-178.410000
25%,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,0.000000,...,NaN,NaN,NaN,1.000000,1.000000,NaN,NaN,NaN,-8.990000,-70.040000
50%,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,0.000000,...,NaN,NaN,NaN,2.000000,1.000000,NaN,NaN,NaN,-1.650000,12.920000
75%,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,0.000000,...,NaN,NaN,NaN,2.000000,1.000000,NaN,NaN,NaN,12.140000,97.740000


In [7]:
avonet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90303 entries, 0 to 90302
Data columns (total 36 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Avibase.ID           90303 non-null  object 
 1   Species1_BirdLife    89966 non-null  object 
 2   Species2_eBird       89939 non-null  object 
 3   eBird.species.group  90122 non-null  object 
 4   Species3_BirdTree    90303 non-null  object 
 5   Data.type            89963 non-null  float64
 6   Source               89956 non-null  object 
 7   Specimen.number      73736 non-null  object 
 8   Sex                  89963 non-null  object 
 9   Age                  89963 non-null  float64
 10  Locality             0 non-null      float64
 11  Country_WRI          77199 non-null  object 
 12  Country              0 non-null      float64
 13  Beak.Length_Culmen   74172 non-null  float64
 14  Beak.Length_Nares    55692 non-null  float64
 15  Beak.Width           70277 non-null 

### Cleaning Data

In [8]:
columns = avonet.columns

# Determine percent of missing data in each column
# Drop columns with 25% or more missing data
for i in columns:
    missing = avonet[i].isnull().sum()/len(avonet)
    if missing >= 0.25:
        avonet.drop(i, axis = 1, inplace = True)

In [9]:
# Drop irrelevant columns
avonet = avonet[['Order', 'Sex', 'Age', 'Beak.Length_Culmen', 'Beak.Width', 'Beak.Depth',
                 'Tarsus.Length', 'Wing.Length', 'Habitat', 'Habitat.Density', 'Migration', 'Trophic.Level',
                 'Trophic.Niche', 'Primary.Lifestyle', 'Centroid.Latitude', 'Centroid.Longitude']]

In [10]:
# Encode "Sex" feature
definitions = {'M':0, 'F':1, 'U':2}
avonet['sex_categorical'] = avonet.Sex.map(definitions)
avonet.drop('Sex', axis = 1, inplace = True)

In [11]:
# Drop rows with missing categorical data
# There are very few missing values for these features, so it won't decrease the dataset much
avonet.dropna(subset=['Habitat', 'Trophic.Level', 'Trophic.Niche', 'Primary.Lifestyle'], inplace = True, 
              ignore_index = True)

In [12]:
# Impute missing numeric values using KNN imputation
# Use only numeric features
avonet_numeric = avonet[['Age', 'Beak.Length_Culmen', 'Beak.Width', 'Beak.Depth', 'Tarsus.Length', 'Wing.Length',
                         'Habitat.Density', 'Migration', 'Centroid.Latitude', 'Centroid.Longitude', 'sex_categorical']]

# Create imputer object
knn_imputer = KNNImputer(n_neighbors = 3)

# Fit and transform numeric data with imputer
avonet_num_trans = knn_imputer.fit_transform(avonet_numeric)

In [13]:
# Convert transformed numeric data to DataFrame
avonet_num_df = pd.DataFrame(avonet_num_trans, columns = ['Age', 'Beak.Length_Culmen', 'Beak.Width', 'Beak.Depth', 
                                                          'Tarsus.Length', 'Wing.Length','Habitat.Density', 'Migration',
                                                          'Centroid.Latitude', 'Centroid.Longitude', 'sex_categorical'])

In [19]:
# Replace original columns with imputed columns
col_names = ['Age', 'Beak.Length_Culmen', 'Beak.Width', 'Beak.Depth', 'Tarsus.Length', 'Wing.Length','Habitat.Density',
             'Migration', 'Centroid.Latitude', 'Centroid.Longitude', 'sex_categorical']

for col in col_names:
    avonet[col] = avonet_num_df[col]

In [20]:
avonet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90135 entries, 0 to 90134
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Order               90135 non-null  object 
 1   Age                 90135 non-null  float64
 2   Beak.Length_Culmen  90135 non-null  float64
 3   Beak.Width          90135 non-null  float64
 4   Beak.Depth          90135 non-null  float64
 5   Tarsus.Length       90135 non-null  float64
 6   Wing.Length         90135 non-null  float64
 7   Habitat             90135 non-null  object 
 8   Habitat.Density     90135 non-null  float64
 9   Migration           90135 non-null  float64
 10  Trophic.Level       90135 non-null  object 
 11  Trophic.Niche       90135 non-null  object 
 12  Primary.Lifestyle   90135 non-null  object 
 13  Centroid.Latitude   90135 non-null  float64
 14  Centroid.Longitude  90135 non-null  float64
 15  sex_categorical     90135 non-null  float64
dtypes: f

### References

1. Schuldheisz, C.: Fish and wildlife service delists 21 species from the endan-
gered species act due to extinction, https://www.fws.gov/press-release/2023-10/21-
species-delisted-endangered-species-act-due-extinction
2. Tobias: ”avonet: morphological, ecological and geographical data for all birds”.
https://figshare.com/s/b990722d72a26b5bfead (2022), accessed on October 18,
2023
3. Tobias, J.A., Sheard, C., Pigot, A.L., Devenish, A.J.M., Yang, J., Sayol, F., Neate-
Clegg, M.H.C., Alioravainen, N., Weeks, T.L., Barber, R.A., Walkden, P.A., Mac-
Gregor, H.E.A., Jones, S.E.I., Vincent, C., Phillips, A.G., Marples, N.M., Monta ̃no-
Centellas, F.A., Leandro-Silva, V., Claramunt, S., Darski, B., Freeman, B.G., Breg-
man, T.P., Cooney, C.R., Hughes, E.C., Capp, E.J.R., Varley, Z.K., Friedman,
N.R., Korntheuer, H., Corrales-Vargas, A., Trisos, C.H., Weeks, B.C., Hanz, D.M.,
T ̈opfer, T., Bravo, G.A., Remeˇs, V., Nowak, L., Carneiro, L.S., Moncada R., A.J.,
Matysiokov ́a, B., Baldassarre, D.T., Mart ́ınez-Salinas, A., Wolfe, J.D., Chapman,
P.M., Daly, B.G., Sorensen, M.C., Neu, A., Ford, M.A., Mayhew, R.J., Fabio Sil-
veira, L., Kelly, D.J., Annorbah, N.N.D., Pollock, H.S., Grabowska-Zhang, A.M.,
McEntee, J.P., Carlos T. Gonzalez, J., Meneses, C.G., Mu ̃noz, M.C., Powell, L.L.,
Jamie, G.A., Matthews, T.J., Johnson, O., Brito, G.R.R., Zyskowski, K., Crates,
R., Harvey, M.G., Jurado Zevallos, M., Hosner, P.A., Bradfer-Lawrence, T., Maley,
J.M., Stiles, F.G., Lima, H.S., Provost, K.L., Chibesa, M., Mashao, M., Howard,
J.T., Mlamba, E., Chua, M.A.H., Li, B., G ́omez, M.I., Garc ́ıa, N.C., P ̈ackert,
M., Fuchs, J., Ali, J.R., Derryberry, E.P., Carlson, M.L., Urriza, R.C., Brzeski,
Bird Classification Model 5
K.E., Prawiradilaga, D.M., Rayner, M.J., Miller, E.T., Bowie, R.C.K., Lafontaine,
R.M., Scofield, R.P., Lou, Y., Somarathna, L., Lepage, D., Illif, M., Neuschulz,
E.L., Templin, M., Dehling, D.M., Cooper, J.C., Pauwels, O.S.G., Analuddin, K.,
Fjelds ̊a, J., Seddon, N., Sweet, P.R., DeClerck, F.A.J., Naka, L.N., Brawn, J.D.,
Aleixo, A., B ̈ohning-Gaese, K., Rahbek, C., Fritz, S.A., Thomas, G.H., Schleuning,
M.: Avonet: morphological, ecological and geographical data for all birds. Ecology
Letters 25(3), 581–597 (2022). https://doi.org/https://doi.org/10.1111/ele.13898,
https://onlinelibrary.wiley.com/doi/abs/10.1111/ele.13898

In [16]:
# Display system data
from watermark import watermark
print(watermark())

Last updated: 2023-10-30T22:06:29.886008-05:00

Python implementation: CPython
Python version       : 3.9.12
IPython version      : 8.2.0

Compiler    : Clang 12.0.0 
OS          : Darwin
Release     : 23.0.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit

